In [ ]:
item_list = ["95", "96", "97", "98", "99"]
classes = ["LETTERS", "CARDINAL", "VERBATIM", "ORDINAL", "DECIMAL", "DIGIT", "MONEY", "FRACTION", "TIME", "ADDRESS"]
counter = {x:0 for x in classes}
desired_samples = 1000
classes_to_sample = classes

written_len = 0
spoken_len = 0

for item in item_list: 
    with open(f"output-000{item}-of-00100", "r") as file:
        sentence_written_list = []
        sentence_spoken_list = []
        sentence_written = ""
        sentence_spoken = ""
        flag = False
        
        for line in file:
            line = line.replace("\n", "")
            fields = line.split("\t")
            if not fields[1].isascii():
                flag = True
            if len(fields)==2:
                if flag or len(sentence_spoken)>=1024:
                    flag = False
                    # print("discarded line: ", sentence_written)
                    sentence_written = ""
                    sentence_spoken = ""
                    continue
                
                temp = ""
                for i, char in enumerate(sentence_written):
                    if char in ",.'-%" and i>0 and sentence_written[i-1]!=" ":
                        temp += " " + char
                    else:
                        temp += char
                    if char in ".," and i<len(sentence_written)-1 and sentence_written[i+1] != " ":
                        temp += " "
                sentence_written = temp
                sentence_written_list.append(sentence_written)
                sentence_spoken_list.append(sentence_spoken)
                sentence_written = ""
                sentence_spoken = ""
            elif len(fields)==3:
                seperator = " "
                # seperator = ""
                # if len(sentence_spoken) != 0 and fields[2] != "sil":
                #     seperator = " "
                ignored_symbols = ['"', "(", ")"]
                if fields[1] in ignored_symbols: # discard ignored symbols
                    continue
                sentence_written += seperator + fields[1]
                if fields[2] == "<self>": # copy words
                    sentence_spoken += seperator + fields[1].casefold()
                elif fields[2] == "sil":
                    pass
                elif fields[0] == "ELECTRONIC":
                    components = fields[2].split(" _letter ")
                    components = [c.replace("_letter", "").replace(" ", "") for c in components]
                    newspoken = " ".join(components).replace("dot", " dot ")
                    sentence_spoken += seperator + newspoken.casefold()
                else:
                    sentence_spoken += seperator + fields[2].casefold()
            
        spoken_len += len(sentence_spoken_list)
        with open("test_spoken.txt", "a") as file:
            for sentence in sentence_spoken_list:
                file.write(sentence+"\n")

        written_len += len(sentence_written_list)
        with open("test_written.txt", "a") as file:
            for sentence in sentence_written_list:
                file.write(sentence+"\n")
            

print("length of spoken sentence list: ", spoken_len)
print("length of written sentence list: ", written_len)

In [ ]:
index_remove = set()

In [ ]:
with open("test_spoken.txt", "r") as file1, open("test_written.txt", "r") as file2:
    for idx, (spoken, written) in enumerate(zip(file1, file2)):
        if len(spoken)>1024:
            print(idx, spoken.replace("\n", ""))
            print(idx, written.replace("\n", ""))
            index_remove.add(idx)

In [ ]:
with open("test_written.txt", "r") as file:
    for idx, line in enumerate(file):
        if len(line)>1500:
            print(idx, line)

In [1]:
with open("test_written.txt", "r") as file:
    print(len(file.readlines()))

4290238


In [2]:
with open("test_spoken.txt", "r") as file:
    print(len(file.readlines()))

4290238


In [3]:
start_idx = 0
end_idx = 89
counter = 0
for idx in range(start_idx, end_idx+1):
    print(f"reading {idx}")
    with open("output-000{:0>2d}-of-00100".format(idx), "r") as file:
        non_ascii_flag = False
        ITN_flag = False
        for line in file:
            line = line.replace("\n", "")
            fields = line.split("\t")
            if not fields[1].isascii():
                non_ascii_flag = True
            if len(fields)==2:
                if non_ascii_flag:
                    non_ascii_flag = False
                elif ITN_flag:
                    counter += 1
                    ITN_flage = False

            elif len(fields)==3:
                if fields[2] != "<self>" and fields[2] != "sil": # copy words
                    ITN_flag = True

reading 0
reading 1
reading 2
reading 3
reading 4
reading 5
reading 6
reading 7
reading 8
reading 9
reading 10
reading 11
reading 12
reading 13
reading 14
reading 15
reading 16
reading 17
reading 18
reading 19
reading 20
reading 21
reading 22
reading 23
reading 24
reading 25
reading 26
reading 27
reading 28
reading 29
reading 30
reading 31
reading 32
reading 33
reading 34
reading 35
reading 36
reading 37
reading 38
reading 39
reading 40
reading 41
reading 42
reading 43
reading 44
reading 45
reading 46
reading 47
reading 48
reading 49
reading 50
reading 51
reading 52
reading 53
reading 54
reading 55
reading 56
reading 57
reading 58
reading 59
reading 60
reading 61
reading 62
reading 63
reading 64
reading 65
reading 66
reading 67
reading 68
reading 69
reading 70
reading 71
reading 72
reading 73
reading 74
reading 75
reading 76
reading 77
reading 78
reading 79
reading 80
reading 81
reading 82
reading 83
reading 84
reading 85
reading 86
reading 87
reading 88
reading 89


In [4]:
counter

77244691

# Formating aisg data into parallel corpus

In [5]:
train_file_path = "AISG/final_train.tsv"
valid_file_path = "AISG/validation.tsv"
test_file_path = "AISG/hard_test_set.tsv"
processed_train_spoken_file_path = "AISG/processed/train_spoken.txt"
processed_train_written_file_path = "AISG/processed/train_written.txt"
processed_valid_spoken_file_path = "AISG/processed/valid_spoken.txt"
processed_valid_written_file_path = "AISG/processed/valid_written.txt"
processed_test_spoken_file_path = "AISG/processed/test_spoken.txt"
processed_test_written_file_path = "AISG/processed/test_written.txt"

In [2]:
# check the symbols and non ascii characters in the dataset
symbols = set()
with open(train_file_path, "r") as train_file, open(valid_file_path, "r") as valid_file, open(test_file_path, "r") as test_file:
    for line in train_file:
        line = line.strip().split("\t")
        text = line[1]
        for char in text:
            if not char.isalnum() or not char.isascii():
                symbols.add(char)
    print(symbols)
    
    for line in valid_file:
        line = line.strip().split("\t")
        text = line[1]
        for char in text:
            if not char.isalnum() or not char.isascii():
                symbols.add(char)
    print(symbols)

    for line in test_file:
        line = line.strip().split("\t")
        text = line[1]
        for char in text:
            if not char.isalnum() or not char.isascii():
                symbols.add(char)
    print(symbols)


{' ', '"', '!', '€', '/', '[', '+', '.', '_', '#', ')', '?', '(', '-', ',', '&', '%', ':', ';', '$', '>', '£', "'", 'µ', '¥', '~', ']', 'Ω', '<'}
{' ', '"', '!', '€', '/', '[', '+', '.', '_', '#', ')', '?', '(', '-', '}', ',', '&', '%', 'α', ':', ';', '$', '>', '£', "'", 'µ', '¥', '~', ']', 'Ω', '<', 'ˈ', '{'}
{' ', '"', '!', '€', '/', '[', '+', '.', '_', '#', ')', '?', '(', '-', '}', ',', '&', '%', 'α', ':', ';', '$', '>', '£', "'", 'µ', '¥', '~', ']', 'Ω', '<', 'ˈ', '{'}


In [10]:
def get_parallel_corpus(input_file, s_file, w_file, sentence_limit=-1):
    symbols_to_remove = "[]{}():\"'<>" # symbols to remove when acting as punctuation
    sentence_written_list = []
    sentence_spoken_list = []
    sentence_written = []
    sentence_spoken = []
    count = 0

    for line in input_file:
        fields = line.strip().split("\t")

        if fields[0] == "<eos>":
            if len(sentence_spoken)>=128:
                sentence_written = []
                sentence_spoken = []
                continue
            
            sentence_written = " ".join(sentence_written)
            sentence_spoken = " ".join(sentence_spoken)
            sentence_written_list.append(sentence_written)
            sentence_spoken_list.append(sentence_spoken)
            sentence_written = []
            sentence_spoken = []

            count += 1
            if sentence_limit!=-1 and count == sentence_limit:
                break

        elif fields[0] == "PUNCT":
            if fields[1] in symbols_to_remove: # discard ignored symbols
                continue
            sentence_written.append(fields[1])
        elif fields[2] == "<self>":
            # copy words 
            sentence_spoken.append(fields[1].casefold())
            sentence_written.append(fields[1])
        else:
            sentence_spoken.append(fields[2])
            sentence_written.append(fields[1])
    
    for sen in sentence_spoken_list:
        s_file.write(sen + "\n")
    for sen in sentence_written_list:
        w_file.write(sen + "\n")

In [11]:
with open(train_file_path, "r") as input_file, \
    open(processed_train_spoken_file_path, "w") as s_file, \
    open(processed_train_written_file_path, "w") as w_file:

    get_parallel_corpus(input_file, s_file, w_file)

with open(valid_file_path, "r") as input_file, \
    open(processed_valid_spoken_file_path, "w") as s_file, \
    open(processed_valid_written_file_path, "w") as w_file:

    get_parallel_corpus(input_file, s_file, w_file, sentence_limit=40000)

with open(test_file_path, "r") as input_file, \
    open(processed_test_spoken_file_path, "w") as s_file, \
    open(processed_test_written_file_path, "w") as w_file:

    get_parallel_corpus(input_file, s_file, w_file)